相比于simple prediction 总体上要修改的不多，只需要

1. 预处理一下 y 把真实收益率处理成rank

2. 修改预测头，现在让模型对每个资产输出一个预测值，按照这个预测值排序

3. 修改损失函数为排序类损失函数

1和2都很简单，但损失函数也是一个问题：排序本身是离散的，如何让离散的损失函数能顺利传播梯度呢？

其实还是相同的思路，将离散分类转为概率，因为就算我们的模型基于目前的信息完美预测了未来趋势，但是未来趋势也会因为未来的新因素，围绕我们的预测波动，或者说，从先验概率偏移到后验概率

In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn


首先只考虑两个资产，假设模型输出的，对资产i,j的强势程度评分score 为Si, Sj 且有 Si > Sj 那么模型当然是预估i资产更强的，但是具体有多强，还是取决于Si 与Sj的相对大小。

换言之，如果Si 只比Sj大一点点，那么其实P(i>j) 应该相当接近0.5，在这种情况下，预测错误的损失应当较小，因为模型相当于是在告诉我们，这两个资产是很接近的，我对资产强弱的相对关系非常不确定

为了防止模型懒惰，对所有资产都输出相同数字，还可以进行归一化

这样就把原始问题转变为了两两计算交叉熵问题。实现如下：

In [2]:
class RankLoss(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.bceloss = nn.BCEWithLogitsLoss() # 用这个版本的BCE可以直接输入logits
    
    
    def forward(self, pred, real):
        """
        向量化加速后的forward
        对每一个资产输出一个分数，pred和real的格式都是(batch_size, num_assets)
        不同的是，real是rank, [r1, r2, ....] pred 是预测的分数
        """
        # 实际输入的real不一定已经转化为了rank，也可能是收益率。我们通过双重索引强制将real转换为rank，就算输入的确实是rank在这次强制转换中并不会发生变化
        sorted_index = torch.argsort(real, dim=1, descending=False)
        rank = torch.argsort(sorted_index, dim = 1)

        # 列出所有的ij组合
        indices = torch.combinations(torch.arange(pred.shape[1]), r=2)
        # 将索引移到与输入张量相同的设备上
        indices = indices.to(pred.device)
        i_indices = indices[:, 0]
        j_indices = indices[:, 1]
        
        # pred[:, i_indices] 的 shape: [batch_size, num_combinations]
        pred_i = pred[:, i_indices]
        pred_j = pred[:, j_indices]
        
        rank_i = rank[:, i_indices]
        rank_j = rank[:, j_indices]

        # prob_diff 的 shape: [batch_size, num_combinations]
        prob_diff = pred_i - pred_j
        
        # real_diff 的 shape: [batch_size, num_combinations]
        real_diff = (rank_i < rank_j).float()

        # 这里我们将 batch_size 和 num_combinations 两个维度展平
        # prob_diff.view(-1) 的 shape: [batch_size * num_combinations]
        return self.bceloss(prob_diff.view(-1), real_diff.view(-1))
    
loss_fn = RankLoss()
pred = torch.randn(size = (1024, 10))
real = torch.randn(size = (1024, 10))
loss_fn(pred, real)

tensor(0.8964)

In [3]:
real = torch.tensor([[0.5,0.2]], dtype = float) # 第一个资产收益更高
sorted_index = torch.argsort(real, dim=1, descending=False)
rank = torch.argsort(sorted_index, dim=1)

pred = torch.tensor([[10,1]], dtype = float) # 第一个资产得分更高（正确排序）

rank, loss_fn(pred, real), loss_fn(pred, rank) # loss很小

(tensor([[1, 0]]), tensor(9.0001), tensor(9.0001))

In [4]:
real = torch.tensor([[0.5,0.2]], dtype = float) # 第一个资产收益更高
sorted_index = torch.argsort(real, dim=1, descending=False)
rank = torch.argsort(sorted_index, dim=1)

pred = torch.tensor([[1,10]], dtype = float) # 第二个资产得分更高（错误排序）

rank, loss_fn(pred, real), loss_fn(pred, rank) # loss很大

(tensor([[1, 0]]), tensor(0.0001), tensor(0.0001))

我们很快就会发现，这个损失函数的效果并不好，因为它对任何ij顺序的错配都是等价看待的。这并不是我们想要的结果：我们关心和交易的是top k 资产，即最强势和最弱势的那一批资产，而对于中间的部分我们不关心

对于一个截面来说，当前周期明显强势和弱势的资产始终是少数，大部分资产其实是集中在均值附近，从而导致这部分本来就更容易错配

换言之，我们最关注的部分，在整个损失函数中只占很小一部分，大部分损失函数的值都被中间的无关噪声占据，从而导致模型反向传播训练的时候非常容易关注到这些噪声导致过拟合。

一个很自然的想法是，基于rank 对loss进行加权，ij 之间的rank差距越大，这次错配的损失也就越大，即对于两个本来就不会交易的资产来说，即使错配了他们的rank也无所谓；而对于两个本来就都在top k 内的资产，错配的损失也不大，毕竟也参与了交易

我们需要惩罚的是对ij差值很大的错配，比如将一个本应该在topk内参与交易的资产错配到了中部，甚至错配到了完全相反的方向，都是应当被加重惩罚的。

对于惩罚的权重，我们希望是：

1. i j 之间的差值越大，此次错配后果更严重，错配损失的权重越大。 loss 应该正比于 (rank i - rank j)^2  或 abs(rank i - rank j)

2. 在i j 差值相同的情况下，差值发生的位置也很关键，越靠近两侧的错配权重也应该更大； ((rank i  + rank j)/2 - mean rank) ^ 2 或 abs(rank i - rank j)，计算此次错配发生的位置是否在中间。例如错配rank排名第1的资产和第10的资产应当是一个严重的结果，因为稍微改变topk就有可能导致资产被移出交易区，而错配第23和第33个资产的惩罚应该小一些，因为在中间区域基本不会被交易，而且中间区域本来被错配的概率本来就很大。

3. 通过超参数 alpha控制这两个weights 的比例，计算出每个ij 对的 weights后归一化，再将这个weights应用在每个单独的 i j pair loss上求加权平均

In [ ]:
class WeightedRankLoss(nn.Module):
    """
    加权排序损失函数，对于排序错配施加不同的权重损失
    """
    def __init__(self, alpha=0.5, p=1, q=2):
        """
        p 控制幅度权重的指数，衡量本次错配的距离，1为绝对差值 2为平方差之
        q 控制位置权重的指数，衡量本次错配发生的位置的重要性，1为绝对差值 2为平方差值
        alpha控制两个权重的比例，alpha越接近1，幅度权重越大
        """
        super().__init__()
        self.alpha = alpha
        self.p = p
        self.q = q
        
        self.bceloss = nn.BCEWithLogitsLoss(reduction='none') # 用这个版本的BCE可以直接输入logits，但现在需要设置reduction='none' 来拿到原始的loss来加权

    def forward(self, pred, real):
        # 通过双重索引强制将real转换为rank
        sorted_indices = torch.argsort(real, dim=1, descending=False)
        rank = torch.argsort(sorted_indices, dim=1)

        # 列出所有的ij组合
        num_assets = pred.shape[1]
        indices = torch.combinations(torch.arange(num_assets), r=2)
        indices = indices.to(pred.device)
        i_indices = indices[:, 0]
        j_indices = indices[:, 1]
        
        # 提取成对的预测分数和真实排名
        pred_i = pred[:, i_indices]
        pred_j = pred[:, j_indices]
        rank_i = rank[:, i_indices].float()
        rank_j = rank[:, j_indices].float()

        # 计算预测得分差和真实目标
        # 我们期望 pred_i > pred_j 当 rank_i < rank_j 时
        pred_diff = pred_i - pred_j
        # target=1.0 表示 i 应该排在 j 前面
        real_diff = (rank_i < rank_j).float()

        # 权重p: 错配的幅度 (Magnitude Weight)
        w_mag = torch.pow(torch.abs(rank_i - rank_j), self.p)
        
        # 权重q: 错配的位置 (Location Weight)
        mean_rank = (num_assets - 1) / 2.0
        mid_point = (rank_i + rank_j) / 2.0
        w_loc = torch.pow(torch.abs(mid_point - mean_rank), self.q)
        
        # 为了避免不同尺度问题，在混合前先对每个样本的权重进行归一化
        w_mag_normalized = w_mag / (w_mag.sum(dim=1, keepdim=True) + 1e-9)
        w_loc_normalized = w_loc / (w_loc.sum(dim=1, keepdim=True) + 1e-9)
        
        # 组合权重
        raw_weights = self.alpha * w_mag_normalized + (1 - self.alpha) * w_loc_normalized
        
        # 对最终权重进行归一化并缩放，使其平均值为1，以保持损失的量级稳定
        num_combinations = i_indices.shape[0]
        final_weights = (raw_weights / (raw_weights.sum(dim=1, keepdim=True) + 1e-9)) * num_combinations

        # 计算原始loss应用自定义权重
        unreduced_loss = self.bceloss(pred_diff, real_diff)
        weighted_loss = unreduced_loss * final_weights

        return weighted_loss.mean()


loss_fn = WeightedRankLoss(alpha=0.5, p=1, q=2)
pred = torch.randn(size=(1024, 50))
real = torch.randn(size=(1024, 50)) # 模拟未来收益率
loss = loss_fn(pred, real)

print(f"Custom Weighted Rank Loss: {loss.item()}")

Custom Weighted Rank Loss: 0.9033253192901611
